You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [9]:
from dotenv import load_dotenv, find_dotenv

find_dotenv()

'e:\\Personal\\blockchain\\GenAi\\Multi-AI-Agent-Systems-with-crewAI-main\\Research_write_article\\.env'

In [10]:
load_dotenv(find_dotenv())

True

In [11]:
import os

os.environ['MY_NAME']

'Areeb Ahmed'

In [15]:
import os

os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

In [ ]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [16]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [17]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [19]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [21]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [22]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [23]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [24]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [25]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [2024-09-01 20:46:42][DEBUG]: == Working Agent: Content Planner
 [2024-09-01 20:46:42][INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer.

Final Answer:

**Comprehensive Content Plan for Blog Article on Artificial Intelligence**

---

**1. Latest Trends, Key Players, and Noteworthy News in Artificial Intelligence:**

- **Trends:**
  - **Generative AI:** Focus on technologies like GPT-3 and DALL-E that create text and images respectively. The rise of AI for content creation and media.
  - **AI in Healthcare:** Usage of AI for diagnostics, personalized medicine, and robotic surgery.
  - **AI Ethics and Regulation:** Gro

- Display the results of your execution as markdown in the notebook.

In [27]:
from IPython.display import Markdown

Markdown(result.raw)

```markdown
# Understanding Artificial Intelligence: Trends, Players, and Future Implications

## Introduction

Artificial Intelligence (AI) has swiftly transitioned from a niche technological concept to a cornerstone of modern society. Defined as the simulation of human intelligence processes by machines, particularly computer systems, AI encompasses a range of capabilities, including learning, reasoning, and self-correction. Its significance is underscored by its pervasive application across various sectors, from healthcare to finance, and its potential to revolutionize how we live and work.

Recent advancements in AI technologies, driven by innovations such as generative AI, have sparked intense interest and debate. As we delve into the latest trends, key players, and the broader implications of AI, it becomes clear that understanding this field is crucial for anyone navigating today’s rapidly evolving landscape.

## Current Trends in AI

### Generative AI

One of the most exciting trends in AI is the rise of generative AI technologies like GPT-3 and DALL-E, which can create human-like text and generate images from textual descriptions, respectively. These tools have opened new avenues for content creation, enabling businesses and individuals to produce high-quality material with minimal effort. The implications of such technology are vast, impacting industries ranging from marketing to entertainment and even education.

Moreover, generative AI fosters creativity by allowing users to explore new concepts and ideas. As this technology evolves, it will likely lead to collaborative partnerships between humans and machines, pushing the boundaries of innovation and artistic expression.

### AI in Healthcare

AI is also making significant strides in the healthcare sector. From diagnostics powered by machine learning to personalized medicine tailored to individual genetic profiles, the potential applications are transformative. Robotic surgeries, guided by AI algorithms, promise greater precision and reduced recovery times. As AI continues to integrate into healthcare, it raises important questions about data privacy and the ethical use of patient information.

Furthermore, AI's ability to analyze vast amounts of data quickly allows for improved patient outcomes through early detection of diseases and more effective treatment plans. As we embrace these advancements, it is essential to remain vigilant about the ethical considerations involved in AI's deployment in healthcare settings.

## Key Players in the AI Landscape

### Leading Companies

Several key players dominate the AI landscape, each contributing uniquely to the field. OpenAI, for instance, is at the forefront of generative models, pushing the boundaries of what AI can achieve. Google DeepMind's work in reinforcement learning has revolutionized how machines learn from their environments, particularly in healthcare applications. IBM Watson focuses on business applications of AI, enhancing operational efficiencies, while NVIDIA provides the hardware critical for AI processing.

Additionally, Microsoft integrates AI into its cloud services, making advanced technologies accessible to a broader audience. These companies not only drive technological advancements but also set industry standards that influence how AI is developed and applied across various sectors.

### Startups and Innovation

Beyond these giants, a wave of AI startups is emerging, focusing on niche markets such as agriculture, environmental conservation, and personalized education. These companies leverage AI to address specific challenges and innovate within their industries, demonstrating the versatility and adaptability of AI technologies.

The agility of startups allows them to experiment with novel applications and solutions that larger corporations may overlook. As they disrupt traditional business models, these startups contribute to a dynamic AI ecosystem that fosters ongoing innovation and competition.

## Impact of AI on Society

### Ethical Considerations

As AI technologies advance, ethical considerations surrounding their development and deployment become increasingly paramount. Issues such as bias in AI algorithms and the need for robust data privacy measures are critical discussions that must be addressed. Companies and governments alike are recognizing the importance of establishing ethical frameworks to ensure AI is developed responsibly and equitably.

Moreover, engaging diverse stakeholders in these conversations is essential for creating inclusive policies. By prioritizing transparency and accountability, the AI community can work towards fostering trust and ensuring that advancements benefit society as a whole.

### Job Displacement and Future of Work

Another pressing concern is the impact of AI on employment. While AI has the potential to enhance productivity and create new job opportunities, it also poses significant risks of job displacement. According to reports from the World Economic Forum, millions of jobs may be affected as industries automate routine tasks. It is essential for policymakers and business leaders to consider strategies for workforce reskilling and adaptation to mitigate these impacts.

Proactive measures, such as investing in education and training programs, can help ensure that the workforce is equipped to thrive in an AI-driven economy. By fostering a culture of lifelong learning, we can prepare individuals for the evolving job market and minimize the disruptions caused by technological advancements.

## Case Studies

### Real-World Applications

Several industries provide compelling case studies of AI implementation. In agriculture, AI-driven tools are optimizing crop yields and reducing waste through predictive analytics and precision farming techniques. In finance, AI algorithms are enhancing fraud detection and streamlining customer service through chatbots. These examples illustrate the tangible benefits of AI, underscoring its potential to drive efficiency and innovation across sectors.

Additionally, the retail industry is leveraging AI for personalized shopping experiences, utilizing data analytics to predict consumer preferences and behavior. By harnessing AI's capabilities, businesses can create more engaging and tailored interactions, ultimately leading to increased customer satisfaction and loyalty.

## Future Directions

### Predictions for the Next Decade

Looking ahead, the next decade promises to be transformative for AI. As quantum computing matures, its intersection with AI could unlock unprecedented capabilities, enabling more complex problem-solving. The ongoing development of AI ethics and regulatory frameworks will also shape the trajectory of AI, ensuring that its benefits are accessible to all while minimizing risks.

As industries adapt to these advancements, collaborative efforts between academia, industry, and government will be crucial in shaping AI's future. By fostering a cooperative environment, we can harness the full potential of AI while addressing the challenges that arise.

## Conclusion

As we navigate the complexities of artificial intelligence, it is imperative to stay informed about the latest trends, key players, and societal implications. The rapid evolution of AI presents both opportunities and challenges, and understanding these dynamics is essential for business professionals, tech enthusiasts, and policymakers alike. To keep abreast of developments in this field, consider subscribing to industry newsletters or following thought leaders on social media. The future of AI is bright, and being informed is the first step toward harnessing its potential.
```

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [28]:
topic = "Benefits of Remote Work"
result = crew.kickoff(inputs={"topic": topic})

 [2024-09-01 20:50:25][DEBUG]: == Working Agent: Content Planner
 [2024-09-01 20:50:25][INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Benefits of Remote Work.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer  
Final Answer: 

**Content Plan Document: Benefits of Remote Work**

---

**1. Latest Trends, Key Players, and Noteworthy News**

- **Trends:**
  - **Increased Adoption:** Remote work has become a standard practice for many organizations. According to a Gartner study, 47% of companies will allow employees to work remotely full-time post-pandemic.
  - **Hybrid Work Models:** Many businesses are adopting hybrid models, combining remote and in-office work to offer flexibility.
  - **Te

In [30]:
Markdown(result.raw)

```markdown
# The Benefits of Remote Work: Embracing a New Era of Employment

The concept of remote work has gained significant traction over the past few years, particularly in the wake of the COVID-19 pandemic. Once seen as a temporary solution, remote work has become an enduring model for many organizations. According to a Gartner study, 47% of companies plan to allow employees to work remotely full-time even after the pandemic subsides. This article explores the myriad benefits of remote work, shedding light on how it enhances productivity, work-life balance, and overall employee satisfaction for both workers and employers alike.

## Increased Flexibility: A Game Changer for Work-Life Balance

One of the most significant advantages of remote work is the flexibility it offers. Employees can tailor their schedules to accommodate personal commitments, whether it's attending a child's school event or managing health appointments. This flexibility fosters a more balanced lifestyle, allowing individuals to prioritize their responsibilities without sacrificing their professional obligations. 

Additionally, the ability to set a personalized work environment can lead to increased motivation. Employees can create a workspace that suits their preferences, whether that’s a quiet home office or a bustling café. This autonomy not only boosts morale but can also translate to improved productivity and creativity, as workers feel more comfortable and in control of their work environment.

## Cost Savings: Financial Benefits for Employees and Employers

Remote work can also yield substantial cost savings for both employees and employers. For employees, reduced commuting costs and lower expenses for work attire can add up to significant savings. According to studies, the average American spends around $2,000 annually on commuting alone—money that can be redirected toward personal savings or family expenses.

Employers also benefit financially from remote work. By reducing the need for physical office space, companies can save on rent, utilities, and maintenance. In fact, studies indicate that organizations with flexible work arrangements can lower their operational costs significantly, allowing them to allocate resources more efficiently.

## Improved Work-Life Balance: Enhancing Employee Well-Being

Remote work has a profound impact on work-life balance, contributing to overall employee happiness and well-being. With the ability to manage their schedules, employees often report feeling less stressed and more fulfilled. This balance is crucial in a world where burnout is increasingly common.

Moreover, organizations that prioritize employee well-being through remote work policies tend to have lower turnover rates. Reports indicate that companies that adopt flexible working arrangements experience higher levels of employee satisfaction and retention. When employees feel that their personal lives are valued, they are more likely to remain loyal to their employers.

## Access to a Global Talent Pool: Diversity and Innovation

Another compelling benefit of remote work is the ability to tap into a global talent pool. Companies no longer need to limit their hiring to local candidates, allowing them to find the best talent regardless of location. This expanded reach not only increases diversity within teams but also brings a variety of perspectives and skills that can drive innovation.

Diverse teams are known to outperform their more homogeneous counterparts, as they can approach problems from multiple angles and generate creative solutions. By embracing remote work, companies can cultivate a more inclusive and dynamic workforce, ultimately leading to enhanced performance and competitiveness in the market.

## Enhanced Productivity: The Remote Work Advantage

Contrary to common misconceptions, remote work can lead to increased productivity. Many remote workers report higher productivity levels due to fewer office distractions and the ability to create a tailored work environment. A study by Buffer found that 77% of remote workers cite higher productivity as one of their primary benefits.

Furthermore, the implementation of collaboration tools like Zoom, Slack, and Microsoft Teams has made remote communication seamless. These platforms enable teams to collaborate effectively, ensuring that projects remain on track even when team members are spread across different locations.

## Environmental Impact: A Sustainable Choice

The shift towards remote work also has significant environmental benefits. With fewer employees commuting to the office, there is a notable reduction in carbon emissions. Additionally, companies that embrace remote work can lower their energy consumption by minimizing the need for large office spaces.

This commitment to sustainability resonates with today’s workforce, particularly younger generations that prioritize environmental responsibility. By adopting remote work policies, organizations can enhance their corporate social responsibility efforts while contributing to a healthier planet.

## Conclusion: Embracing the Future of Work

The benefits of remote work are manifold, extending beyond mere convenience to encompass financial savings, improved well-being, and environmental sustainability. As both employees and employers adapt to this new paradigm, it is crucial to recognize the positive impacts of flexible working arrangements. 

Organizations that embrace remote work not only foster a happier, more productive workforce but also position themselves as forward-thinking leaders in their industries. Whether you are an employee seeking a better work-life balance or an employer looking to enhance your company's culture, the time to explore the benefits of remote work is now. Evaluate your remote work policies and consider how they can contribute to a more satisfying and productive work environment for everyone involved.
```

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).